This notebook applies a stateful LSTM model using PyTorch & Lightning, to get multi-step quantile energy consumption forecasts. 
\
It also performs & demonstrates the necessary data handling & preprocessing steps to get input & output sequences for the LSTM model.

In [1]:
%pip install numpy==2.0.2 pandas==2.2.2 optuna==4.4.0

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import sys

# 정확한 프로젝트 루트 경로 설정
project_root = os.path.abspath("c:/Users/Daniel/DeepLearningEnergyForecasting/")  # notebooks 폴더 바로 위가 루트

# 확인
print("루트 경로로 이동:", project_root)
os.chdir(project_root)

# sys.path에 추가
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# 다시 확인
print("📂 현재 작업 디렉토리:", os.getcwd())
print("📁 src 경로 존재 여부:", os.path.exists("src/deep_learning"))  # 직접 src 추가
src_path = os.path.join(project_root, "src")
if src_path not in sys.path:
    sys.path.insert(0, src_path)


루트 경로로 이동: c:\Users\Daniel\DeepLearningEnergyForecasting
📂 현재 작업 디렉토리: c:\Users\Daniel\DeepLearningEnergyForecasting
📁 src 경로 존재 여부: True


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import lightning as pl 
#import pytorch_lightning as pl
from optuna.integration import PyTorchLightningPruningCallback

import optuna
import warnings

from lightning.pytorch.callbacks import Callback
#from pytorch_lightning.callbacks import EarlyStopping
from optuna.integration import PyTorchLightningPruningCallback
from src.deep_learning.preprocessing import SequenceScaler, SequenceDataset
from src.deep_learning.quantile_loss import QuantileLoss
from src.deep_learning.lstm import StatefulQuantileLSTM
from src.deep_learning.testing import train_val_split, train_val_split_const_batch_size, test_sequences_to_dataframe, plot_actual_predicted, plot_sequence_preds, calculate_metrics
from src.deep_learning.prediction import predictions_to_dataframe

c:\Users\Daniel\anaconda3\envs\aiot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\Daniel\anaconda3\envs\aiot\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\Daniel\anaconda3\envs\aiot\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "c:\Users\Dani

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
random_seed = 1923

In [6]:
# Set Torch settings
torch.set_default_dtype(torch.float32)
torch.set_float32_matmul_precision('medium')
pl.seed_everything(random_seed, workers = True)
warnings.filterwarnings("ignore", ".*does not have many workers.*")

Global seed set to 1923


In [7]:
# Plot settings
plt.rcParams["figure.autolayout"] = True
plt.rcParams['figure.dpi'] = 100
sns.set_style("darkgrid")

In [8]:
data_dir = "C:/Users/Daniel/DeepLearningEnergyForecasting/data/analysis/processed/"

In [9]:
df = pd.read_csv(data_dir + "train_advanced_featured.csv")
df["일시"] = pd.to_datetime(df["일시"])

In [10]:
df

,건물번호,일시,기온(°C),강수량(mm),풍속(m/s),습도(%),일조(hr),일사(MJ/m2),전력소비량(kWh),연면적(m2),...,건물유형_학교,건물유형_호텔,consumption_lag_24h,trend,hour_sin,hour_cos,day_of_week_sin,day_of_week_cos,month_sin,month_cos
0,1,2024-06-02 00:00:00,18.7,0.0,2.9,58.0,0.0,0.00,3870.42,82912.71,...,False,True,5794.80,24,0.000000,1.000000,-0.781831,0.623490,1.224647e-16,-1.0
1,1,2024-06-02 01:00:00,18.2,0.0,3.5,60.0,0.0,0.00,5733.45,82912.71,...,False,True,5591.85,25,0.258819,0.965926,-0.781831,0.623490,1.224647e-16,-1.0
2,1,2024-06-02 02:00:00,17.6,0.0,2.5,61.0,0.0,0.00,5689.17,82912.71,...,False,True,5338.17,26,0.500000,0.866025,-0.781831,0.623490,1.224647e-16,-1.0
3,1,2024-06-02 03:00:00,16.9,0.0,2.5,64.0,0.0,0.00,5579.04,82912.71,...,False,True,4554.42,27,0.707107,0.707107,-0.781831,0.623490,1.224647e-16,-1.0
4,1,2024-06-02 04:00:00,16.1,0.0,1.6,66.0,0.0,0.00,4113.60,82912.71,...,False,True,3602.25,28,0.866025,0.500000,-0.781831,0.623490,1.224647e-16,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203971,100,2024-08-24 19:00:00,29.1,0.0,4.4,76.0,0.4,0.18,3276.00,162070.24,...,False,True,2206.92,203995,-0.965926,0.258819,-0.974928,-0.222521,-8.660254e-01,-0.5
203972,100,2024-08-24 20:00:00,28.6,0.0,3.7,74.0,0.0,0.00,3197.52,162070.24,...,False,True,2064.12,203996,-0.866025,0.500000,-0.974928,-0.222521,-8.660254e-01,-0.5
203973,100,2024-08-24 21:00:00,28.3,0.0,2.9,74.0,0.0,0.00,3006.60,162070.24,...,False,True,1161.12,203997,-0.707107,0.707107,-0.974928,-0.222521,-8.660254e-01,-0.5
203974,100,2024-08-24 22:00:00,28.0,0.0,1.7,76.0,0.0,0.00,2649.72,162070.24,...,False,True,2076.36,203998,-0.500000,0.866025,-0.974928,-0.222521,-8.660254e-01,-0.5


## Data prep: Getting input & output sequences

We will create a "shifted dataset" where each row at time T contains the following columns:
- Target value at T (consumption lead T+1),
- Past value at T (consumption lag T-2),
- Time covariates at T.

Every input sequence will be the last 72 hours before 16:00, and every output sequence will be the next 32 hours after 16:00.

In [11]:
# 1. Define forecasting and modeling constants
INPUT_LENGTH = 72         # 과거 시계열 길이 (T-L ~ T-1)
OUTPUT_LENGTH = 168       # 미래 예측 길이 (T ~ T+167)
HORIZON_START = 0         # 예측 시점 중 loss를 계산할 첫 시점
QUANTILES = [0.025, 0.5, 0.975]  # 예측할 분위수
FORECAST_HOUR = 16        # 매일 예측을 시작할 시각 (예: 오후 4시)
batch_size = 64  # 🔥 여기에 명시적으로 정의
train_fraction = 0.8

# 2. Feature 정의
FEATURE_COLS = [
    "consumption_lag_24h", "trend", "hour_sin", "hour_cos",
    "day_of_week_sin", "day_of_week_cos", "month_sin", "month_cos"
]

STATIC_COLS = [col for col in df.columns if "건물유형_" in col or "면적" in col or "용량" in col]

TARGET_COL = "전력소비량(kWh)"


In [12]:
# 3. 전체 건물에 대해 시퀀스를 생성하는 함수 정의
def make_sequences_all_buildings(
    df: pd.DataFrame,
    input_length: int = INPUT_LENGTH,
    output_length: int = OUTPUT_LENGTH,
    forecast_hour: int = FORECAST_HOUR
):
    input_seqs = []
    output_seqs = []

    all_static_cols = STATIC_COLS.copy()

    for building_id, group in df.groupby("건물번호"):
        group = group.sort_values("일시").reset_index(drop=True)
        group["시간"] = group["일시"].dt.hour

        # 누락된 정적 변수 채우기
        for col in all_static_cols:
            if col not in group.columns:
                group[col] = 0

        # 예측 가능한 시간 찾기
        forecast_points = group[
            (group["시간"] == forecast_hour)
            & (group.index >= input_length)
            & (group.index + output_length <= group.index.max())
        ].index

        for t in forecast_points:
            input_seq = pd.concat([
                group.loc[t - input_length:t - 1, ["일시"]],
                group.loc[t - input_length:t - 1, FEATURE_COLS],
            ], axis=1).copy()
            for col in all_static_cols:
                input_seq[col] = group[col].iloc[0]
            input_seq = input_seq.set_index("일시")

            output_seq = pd.concat([
                group.loc[t:t + output_length - 1, ["일시"]],
                group.loc[t:t + output_length - 1, [TARGET_COL]],
                group.loc[t:t + output_length - 1, FEATURE_COLS],  # ✅ 이 부분 수정
            ], axis=1).copy()
            for col in all_static_cols:
                output_seq[col] = group[col].iloc[0]
            output_seq = output_seq.set_index("일시")
        
            input_seqs.append((building_id, input_seq))     # tuple로 저장
            output_seqs.append((building_id, output_seq))
            

    return input_seqs, output_seqs


In [13]:
# 4. 시퀀스 생성 실행
input_sequences, output_sequences = make_sequences_all_buildings(df)

print(f"총 시퀀스 수: {len(input_sequences)}")
display(input_sequences[0][1].head())

총 시퀀스 수: 7499


,consumption_lag_24h,trend,hour_sin,hour_cos,day_of_week_sin,day_of_week_cos,month_sin,month_cos,연면적(m2),냉방면적(m2),...,건물유형_IDC(전화국),건물유형_건물기타,건물유형_공공,건물유형_백화점,건물유형_병원,건물유형_상용,건물유형_아파트,건물유형_연구소,건물유형_학교,건물유형_호텔
일시,,,,,,,,,,,,,,,,,,,,,
2024-06-02 16:00:00,4659.69,40,-0.866025,-5.000000e-01,-0.781831,0.62349,1.224647e-16,-1.0,82912.71,77586.0,...,False,False,False,False,False,False,False,False,False,True
2024-06-02 17:00:00,5358.84,41,-0.965926,-2.588190e-01,-0.781831,0.62349,1.224647e-16,-1.0,82912.71,77586.0,...,False,False,False,False,False,False,False,False,False,True
2024-06-02 18:00:00,5985.18,42,-1.000000,-1.836970e-16,-0.781831,0.62349,1.224647e-16,-1.0,82912.71,77586.0,...,False,False,False,False,False,False,False,False,False,True
2024-06-02 19:00:00,6292.32,43,-0.965926,2.588190e-01,-0.781831,0.62349,1.224647e-16,-1.0,82912.71,77586.0,...,False,False,False,False,False,False,False,False,False,True
2024-06-02 20:00:00,5279.67,44,-0.866025,5.000000e-01,-0.781831,0.62349,1.224647e-16,-1.0,82912.71,77586.0,...,False,False,False,False,False,False,False,False,False,True


In the input sequence, we pair every past target value at T with the future covariates at T+1.
\
In the output sequence, we pair every future target value at T+1 with the future covariates at T+2.

This is because the future target at T+1 and the future covariates at T+2 will be the past target & future covariates in the next prediction step. 
\
LSTMs and RNNs can only forecast 1 step at a time as they need hidden & cell states from the previous step. 
\
For validation & prediciton steps, we will replace the future targets after T+1 with predicitons from the previous step, as these will be unknown values at real prediciton time.

During training, we still use the real target values for all prediciton steps "in hindsight", as training with predictions as the target may mislead the model. In a real life scenario, we'd have the "hindsight" values available in the historic data, just like we do here.

In [14]:
len(input_sequences)

7499

In [15]:
len(output_sequences)

7499

## Preprocessing: Custom scaler, Torch datasets & dataloaders

In a stateful LSTM, the last timestep's hidden & cell state for observation 1 in batch 1 will be used as the initial hidden & cell state for observation 1 in batch 2.

Because of this, we need batches of constant size through training, val. and testing data. We'll divide the lengths of each data fold with the batch size, drop the remainder from the start of the training fold & the end of the val. & test folds.
\
The data splitter is implemented in `src.deep_learning.testing`.

In [16]:
# 전체의 0.8을 학습 + 튜닝용으로 쓰고 싶다면:
train_input, train_output, test_input, test_output = train_val_split_const_batch_size(
    input_sequences, output_sequences,
    train_fraction=0.8,  # 전체 중 학습+튜닝
    batch_size=64
)

# 그 중 튜닝용은 다시 0.9 정도로 나누는 게 적당함:
tr_input, tr_output, val_input, val_output = train_val_split_const_batch_size(
    train_input,
    train_output,
    train_fraction=0.9,   # 전체가 아니라 train의 일부만 val로
    batch_size=64
)

We have to scale the past consumption & trend values in the input sequences, and the future consumption & trend values in the output sequences, because they'll be the past values as the forecast horizon expands. We also need the ability to backtransform the network's final predictions accordingly.

This can be done with the custom `SequenceScaler` implemented in `src.deep_learning.preprocessing`.

The scaled data is then converted into Torch tensors using `SequenceDataset`.

In [17]:
#훈련 시퀀스 중 길이 또는 컬럼 수가 잘못된 것 제거
expected_input_shape = (INPUT_LENGTH, len(FEATURE_COLS) + len(STATIC_COLS))
expected_output_shape = (OUTPUT_LENGTH, len(FEATURE_COLS) + 1 + len(STATIC_COLS))

bad_indices = [
    i for i, (x, y) in enumerate(zip(tr_input, tr_output))
    if x.shape != expected_input_shape or y.shape != expected_output_shape
]

from collections import Counter

input_shapes = [x.shape for x in tr_input]
output_shapes = [y.shape for y in tr_output]

print("🔍 Input 시퀀스 shape 분포:", Counter(input_shapes))
print("🔍 Output 시퀀스 shape 분포:", Counter(output_shapes))


print(f"제거될 잘못된 시퀀스 개수: {len(bad_indices)}")

# 안전하게 역순으로 삭제
for i in sorted(bad_indices, reverse=True):
    del tr_input[i]
    del tr_output[i]


🔍 Input 시퀀스 shape 분포: Counter({(72, 23): 5312})
🔍 Output 시퀀스 shape 분포: Counter({(168, 24): 5312})
제거될 잘못된 시퀀스 개수: 0


In [18]:
# 4. 시퀀스 스케일러 (min-max) - val용
scaler_val = SequenceScaler()
_ = scaler_val.fit(tr_input, tr_output)

tr_data = SequenceDataset(
    scaler_val.transform(tr_input), 
    scaler_val.transform(tr_output)
)
val_data = SequenceDataset(
    scaler_val.transform(val_input), 
    scaler_val.transform(val_output)
)

# 5. 시퀀스 스케일러 - test용
scaler_test = SequenceScaler()
_ = scaler_test.fit(train_input, train_output)

train_data = SequenceDataset(
    scaler_test.transform(train_input), 
    scaler_test.transform(train_output)
)
test_data = SequenceDataset(
    scaler_test.transform(test_input), 
    scaler_test.transform(test_output)
)

# 6. Torch Dataloaders 설정
num_workers = 0  # 시스템 코어 수에 따라 조정
pin_memory=True  # GPU 성능 최적화
shuffle = False

tr_loader = torch.utils.data.DataLoader(tr_data, batch_size=batch_size, num_workers=num_workers, shuffle=shuffle, pin_memory=pin_memory)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, num_workers=num_workers, shuffle=shuffle, pin_memory=pin_memory)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, num_workers=num_workers, shuffle=shuffle, pin_memory=pin_memory)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, num_workers=num_workers, shuffle=shuffle, pin_memory=pin_memory)

## Model definition

The `StatefulQuantileLSTM` model, and the `QuantileLoss` function are both implemented in `src.deep_learning`.

## Hyperparameter tuning with Optuna

In [19]:
# Define tuning parameters
tol = 0.002 # Change in MAE to avoid early stopping
patience = 5 # N. of rounds with no improvement before early stopping

In [20]:
# Define Optuna objective
def objective_lstm(trial):
    output_length = OUTPUT_LENGTH
    input_dims = len(FEATURE_COLS) + len(STATIC_COLS)
    horizon_start = HORIZON_START
    quantiles = QUANTILES

    # Define search space
    learning_rate = trial.suggest_float("learning_rate", 5e-4, 5e-2) # 0.0005 to 0.05
    lr_decay = trial.suggest_float("lr_decay", 0.9, 1)
    num_layers = trial.suggest_int("num_layers", 1, 3)
    hidden_size = 2 ** trial.suggest_int("hidden_size_pw", 2, 6) # 4 to 64, powers of 2
    dropout_rate = trial.suggest_float("dropout_rate", 0, 0.1)

    # Create hyperparameters dict
    hyperparameters_dict = {
        "output_length": output_length,
        "input_size": input_dims,
        "horizon_start": horizon_start,
        "quantiles": quantiles,
        "learning_rate": learning_rate,
        "lr_decay": lr_decay,
        "num_layers": num_layers,
        "hidden_size": hidden_size,
        "dropout_rate": dropout_rate
    }

    # Create early stop callback
    callback_earlystop = pl.pytorch.callbacks.EarlyStopping(
        monitor = "val_loss",
        mode = "min",
        min_delta = tol,
        patience = patience
    )

    # Create pruning callback
    callback_pruner = PyTorchLightningPruningCallback(trial, monitor = "val_loss")

    # Create trainer
    trainer = pl.Trainer(
        max_epochs=50,
        detect_anomaly=False,
        accelerator="gpu",     
        devices=1,
        precision="32",     
        callbacks=[callback_earlystop, callback_pruner],
        enable_model_summary=False,
        logger=False,
        enable_progress_bar=True,  # ← True로 바꿔도 무방 (진행상황 보기용)
        enable_checkpointing=False
    )


    # Model
    model = StatefulQuantileLSTM(hyperparameters_dict)
    model.to('cuda')  # ✅ 직접 GPU로 이동

    # 확인용
    print("📦 Model device:", next(model.parameters()).device)

    # 배치도 확인
    sample_batch = next(iter(tr_loader))
    print("📂 Batch device:", sample_batch[0].device)

    # 학습
    trainer.fit(model, tr_loader, val_loader)

    # 평가 결과
    score = callback_earlystop.best_score.cpu().numpy()
    epoch = trainer.current_epoch - callback_earlystop.wait_count
    trial.set_user_attr("n_epochs", epoch)

    # ✅ 자동 저장 (함수 밖에 study_lstm 있어야 함)
    import pickle
    try:
        with open("c:/Users/Daniel/DeepLearningEnergyForecasting/study_lstm.pkl", "wb") as f:
            pickle.dump(study_lstm, f)
        print("💾 study_lstm 저장 완료.")
    except Exception as e:
        print("⚠️ study_lstm 저장 실패:", e)

    return score

In [21]:
# Create study
study_lstm = optuna.create_study(
  sampler = optuna.samplers.TPESampler(seed = random_seed),
  pruner = optuna.pruners.HyperbandPruner(),
  study_name = "tune_lstm",
  direction = "minimize"
)

[I 2025-07-27 16:36:21,993] A new study created in memory with name: tune_lstm


In [22]:
import torch
print(torch.__version__)
print(torch.version.cuda)
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")
import os
print(os.cpu_count())  # 논리적 코어 수 (하이퍼스레딩 포함)


1.13.0
11.6
True
1
NVIDIA GeForce RTX 3050 Ti Laptop GPU
12


In [23]:
'''try:
    study_lstm.optimize(objective_lstm, n_trials=50, show_progress_bar=True)
except KeyboardInterrupt:
    print("중단됨! Study 저장 중...")
    with open(file_path.replace('.pkl', '_new.pkl'), "wb") as f:
        pickle.dump(study_lstm, f)'''

'try:\n    study_lstm.optimize(objective_lstm, n_trials=50, show_progress_bar=True)\nexcept KeyboardInterrupt:\n    print("중단됨! Study 저장 중...")\n    with open(file_path.replace(\'.pkl\', \'_new.pkl\'), "wb") as f:\n        pickle.dump(study_lstm, f)'

In [24]:
# 재시작 후, 다시 불러와서 이어서 할 때
import pickle
with open("c:/Users/Daniel/DeepLearningEnergyForecasting/study_lstm.pkl", "rb") as f:
    study_lstm = pickle.load(f)

# 그리고 다시
study_lstm.optimize(objective_lstm, n_trials=50 - len(study_lstm.trials), show_progress_bar = True)

  0%|          | 0/40 [00:00<?, ?it/s]GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


📦 Model device: cuda:0
📂 Batch device: cpu
Epoch 1: 100%|██████████| 92/92 [05:39<00:00,  3.69s/it, loss=10.4, train_loss_step=4.200, val_loss_step=5.260, val_loss_epoch=6.060, train_loss_epoch=11.60]

  0%|          | 0/40 [10:57<?, ?it/s]

[I 2025-07-27 16:47:22,357] Trial 10 pruned. Trial was pruned at epoch 1.
Epoch 1: 100%|██████████| 92/92 [05:39<00:00,  3.69s/it, loss=10.4, train_loss_step=4.200, val_loss_step=5.260, val_loss_epoch=6.060, train_loss_epoch=11.60]

Best trial: 0. Best value: 3.25528:   2%|▎         | 1/40 [10:57<7:07:29, 657.69s/it]GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


...

[W 2025-07-27 16:47:23,281] Trial 11 failed with parameters: {'learning_rate': 0.027978459117020908, 'lr_decay': 0.9595647624706808, 'num_layers': 3, 'hidden_size_pw': 6, 'dropout_rate': 0.03476351032952749} because of the following error: OutOfMemoryError('CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 4.00 GiB total capacity; 3.45 GiB already allocated; 0 bytes free; 3.46 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF').
Traceback (most recent call last):
  File "c:\Users\Daniel\anaconda3\envs\aiot\lib\site-packages\optuna\study\_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_42176\830938251.py", line 66, in objective_lstm
    trainer.fit(model, tr_loader, val_loader)
  File "c:\Users\Daniel\anaconda3\envs\aiot\lib\site-packages\lightning

Best trial: 0. Best value: 3.25528:   2%|▎         | 1/40 [10:58<7:07:29, 657.69s/it]

[W 2025-07-27 16:47:23,303] Trial 11 failed with value None.
Epoch 0:   0%|          | 0/92 [00:00<?, ?it/s]3.70s/it, loss=10.4, train_loss_step=4.200, val_loss_step=5.260, val_loss_epoch=6.060, train_loss_epoch=11.60]

Best trial: 0. Best value: 3.25528:   2%|▎         | 1/40 [10:58<7:08:06, 658.64s/it]


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 4.00 GiB total capacity; 3.45 GiB already allocated; 0 bytes free; 3.46 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
import os
import pandas as pd

# ✅ 통일된 경로 설정
project_root = "C:/Users/Daniel/DeepLearningEnergyForecasting"
log_path = os.path.join(project_root, "data", "tuning_logs")
os.makedirs(log_path, exist_ok=True)

# ✅ 저장
export_trial_no = "1"
trials_lstm = study_lstm.trials_dataframe().sort_values("value", ascending=True)
trials_lstm.to_csv(os.path.join(log_path, f"trials_lstm{export_trial_no}.csv"), index=False)


## Model testing

In [ ]:
# ✅ 불러오기
import_trial_no = "1"
best_trial_lstm = pd.read_csv(os.path.join(log_path, f"trials_lstm{import_trial_no}.csv")).iloc[0]

In [ ]:
best_trial_lstm

In [ ]:
# Create hyperparameters dict
hyperparameters_dict = {
    "output_length": output_length,
    "input_size": input_dims,
    "horizon_start": horizon_start,
    "quantiles": quantiles,
    "learning_rate": best_trial_lstm["params_learning_rate"],
    "lr_decay": best_trial_lstm["params_lr_decay"],
    "num_layers": best_trial_lstm["params_num_layers"],
    "hidden_size": int(2 ** best_trial_lstm["params_hidden_size_pw"]),
    "dropout_rate": best_trial_lstm["params_dropout_rate"]
}

In [ ]:
best_trial_id = int(best_trial_lstm["number"])
best_trial_full = study_lstm.trials[best_trial_id]  # Optuna Trial 객체

n_epochs = best_trial_full.user_attrs.get("n_epochs", 30)  # 없으면 30

trainer = pl.Trainer(
    max_epochs=n_epochs,
    accelerator="gpu",
    devices=1,
    precision=32,
    enable_model_summary=True,
    logger=False,
    enable_progress_bar=False,
    enable_checkpointing=False
)


In [ ]:
import torch
torch.autograd.set_detect_anomaly(True)
# Create & train model
model = StatefulQuantileLSTM(hyperparameters_dict)
trainer.fit(model, train_loader)

In [ ]:
# Make predictions on test data & backtransform scaled values
preds_raw = trainer.predict(model, test_loader)
preds_raw = torch.cat(preds_raw, dim = 0).cpu().numpy().astype(np.float32)
preds = scaler_test.backtransform_preds(preds_raw)

In [ ]:
# Combine back prediction sequences with times
df_preds = predictions_to_dataframe(preds, test_output)

In [ ]:
df_preds

In [ ]:
# Combine back testing output sequences with input sequences & times
df_test = test_sequences_to_dataframe(test_input, test_output)

In [ ]:
df_test

In [ ]:
# Predicted vs. actual plot, hourly, entire test period
_ = plot_actual_predicted(df_test, df_preds, "Stateful LSTM")

In [ ]:
# Predicted vs. actual plots, zoomed in
indices = [i for i in range(0, len(test_output), len(test_output) // 4)]
indices.append(len(test_output) - 1)
for idx in indices:
    _ = plot_sequence_preds(preds, test_input, test_output, "Stateful LSTM", sequence_index = idx)

In [ ]:
# Calculate performance metrics: MAE, RMSLE, MAPE, pinball loss
calculate_metrics(df_test, df_preds, "Stateful LSTM")

Keep in mind that for the median quantile, pinball loss equals MAE / 2.